# Dataset Creation for Fresh Tracks

# Statistical Distributions of Manually Generated Fake Data

## Exploratory Data Analysis (EDA)

This notebook shall visualize time-series distributions for the following **Company Name** / **Project Name** combinations:

<ol>
    <li>Squirrel Co. Inc. / Project Helios</li>
    <li>Pie Inc. / Project Curve</li>
    <li>Deja &amp; Co. Inc. / Mother's Day 2023</li>
    <li>Squirrel Co. Inc. / Project May'23 BiMo</li>
</ol>

For their respective splits and the following metrics (**Event Type**):

<ol>
    <li>st mail scan</li>
    <li>ID emails sent</li>
    <li>ID emails opened</li>
    <li>ID clickthrough</li>
    <li>ID clickthrough 30Day</li>
    <li>QR Code Scan</li>
    <li>Responses</li>
</ol>

<a id=toc></a>
## Table of Contents

<ul>
    <li><a href=#01-import-packages>Import Packages</a>
    <li><a href=#02-load-dataset>Data Wrangling and Visualization</a>
        <ul>
            <li><a href=#02-a-check-missing>Check Missingness</a>
            <li><a href=#02-b-preprocessing>Preprocessing</a>
            <li><a href=#02-c-visualization>Preliminary Visualization</a>
        </ul>
    <li><a href=#03-save-file>Save Processed File</a>
</ul>

<a id=01-import-packages></a>
## Import Packages

Import necessary packages.

In [1]:
# Apache parquet files (to save space)
# import pyarrow as pa
# import pyarrow.parquet as pq

# Dataframes and numerical
import pandas as pd
import numpy as np

# Increase pandas default display 
pd.options.display.max_rows = 250
pd.options.display.max_columns = 250

# Suppress scientific notation of data in pandas
pd.set_option('display.float_format', '{:.2f}'.format)

# Graphing
# From https://stackoverflow.com/questions/43529054/how-to-plot-pivot-chart-in-python
from pivottablejs import pivot_ui
# import plotly.express as px
# import plotly.graph_objects as go

# Machine learning
# from sklearn.model_selection import GridSearchCV, TimeSeriesSplit             
# from sklearn.linear_model import Lasso, Ridge, ElasticNet
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

<a href=#toc>Back to the top</a>

<a id=02-load-dataset></a>
## Data Wrangling and Visualization

Load **csv** file for training data set.

In [2]:
# Use this method since there is a problem with the filepath in the notebook file
# More here: https://www.youtube.com/watch?v=29vM232xhAk

# Manual fake data
file_1 = '/Users/jchatterjee/Library/CloudStorage/OneDrive-Personal/Documents/NYCDSA/Projects/Upwork/8 - Snowball Print Marketing Dataset Creation/1 - Visualization of Supplied Data/Originals/Fresh Tracks 1.0 Sample Data 3_31 - sample date driven data.csv'

# Company Name / Project Name
file_2 = '/Users/jchatterjee/Library/CloudStorage/OneDrive-Personal/Documents/NYCDSA/Projects/Upwork/8 - Snowball Print Marketing Dataset Creation/1 - Visualization of Supplied Data/Originals/Fresh Tracks 1.0 Sample Data 3_31 - Web Form Data.csv'

data = pd.read_csv(file_1)
copr = pd.read_csv(file_2)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4700 entries, 0 to 4699
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   SB Project #  4700 non-null   object
 1   SB Job #      4700 non-null   int64 
 2   Split         4700 non-null   object
 3   Date          4700 non-null   object
 4   Value         4700 non-null   int64 
 5   State         4700 non-null   object
 6   Event Type    4700 non-null   object
dtypes: int64(2), object(5)
memory usage: 257.2+ KB


In [4]:
data.head()

,SB Project #,SB Job #,Split,Date,Value,State,Event Type
0,P259,572855,US330,2023-03-24,35,DE,st mail scan
1,P259,572855,US330,2023-03-24,164,MA,st mail scan
2,P259,572855,US330,2023-03-24,19,NM,st mail scan
3,P259,572855,US330,2023-03-24,665,OK,st mail scan
4,P259,572855,US330,2023-03-24,91,TN,st mail scan


In [5]:
data.shape

(4700, 7)

In [6]:
# Describe data
data.describe()

,SB Job #,Value
count,4700.00,4700.00
mean,576289.11,602.27
std,12821.08,3510.81
min,535467.00,0.00
25%,572855.00,4.00
50%,581785.00,34.00
75%,581785.00,329.25
max,581785.00,189010.00


In [7]:
copr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   SB Project #         4 non-null      object
 1   SB Job #             4 non-null      int64 
 2   Company Name         4 non-null      object
 3   Project Name         4 non-null      object
 4   # Creative Versions  4 non-null      int64 
 5   Planned Mail Qty     4 non-null      object
 6   Planned Mail Date    4 non-null      object
 7   CPP                  4 non-null      object
dtypes: int64(2), object(6)
memory usage: 384.0+ bytes


In [8]:
copr.head()

,SB Project #,SB Job #,Company Name,Project Name,# Creative Versions,Planned Mail Qty,Planned Mail Date,CPP
0,P259,572855,Squirrel Co Inc,Project Helios,2,"25,000",3/1/23,$0.69
1,P944,535467,Pie Inc,Project Curve,1,"30,000",4/5/23,$0.82
2,P1075,578963,Deja & Co Inc,Mother's Day 2023,1,"10,000",4/10/23,$0.89
3,P1021,581785,Squirrel Co Inc,Project May'23 BiMo,8,"2,377,500",5/4/23,$0.42


<a href=#toc>Back to the top</a>

<a id=02-a-check-missing></a>
### Check Missingness

In [9]:
for col in data.columns:
    print(f'{col}: {data[col].isnull().sum()}')

SB Project #: 0
SB Job #: 0
Split: 0
Date: 0
Value: 0
State: 0
Event Type: 0


<a href=#toc>Back to the top</a>

<a id=02-b-preprocessing></a>
### Preprocessing

Perform the following modifications:

<ol>
    <li>Join DataFrame containing <b>Company Name</b> and <b>Project Name</b> to that of the fake data</li>
    <li>Drop <b>SB Project #</b> and <b>SB Job #</b> columns and create new dataframe with following columns in this order:
        <ol>
            <li><b>Company Name</b></li>
            <li><b>Project Name</b></li>
            <li><b>Split</b></li>
            <li><b>Date</b></li>
            <li><b>Value</b></li>
            <li><b>State</b></li>
            <li><b>Event Type</b></li>
        </ol>
    </li>
</ol>

In [10]:
# Merge DataFrames
data = data.merge(copr, on = 'SB Project #')

# Reindex columns for readability.
col_names = ['Company Name', 'Project Name', 'Split', 'Date', 'Value', 'State', 'Event Type']
data = data.reindex(columns = col_names)

In [11]:
data.head()

,Company Name,Project Name,Split,Date,Value,State,Event Type
0,Squirrel Co Inc,Project Helios,US330,2023-03-24,35,DE,st mail scan
1,Squirrel Co Inc,Project Helios,US330,2023-03-24,164,MA,st mail scan
2,Squirrel Co Inc,Project Helios,US330,2023-03-24,19,NM,st mail scan
3,Squirrel Co Inc,Project Helios,US330,2023-03-24,665,OK,st mail scan
4,Squirrel Co Inc,Project Helios,US330,2023-03-24,91,TN,st mail scan


<a href=#toc>Back to the top</a>

<a id=02-c-visualization></a>
### Preliminary Visualization

In [12]:
# From https://stackoverflow.com/questions/43529054/how-to-plot-pivot-chart-in-python
pivot_ui(data)

<a href=#toc>Back to the top</a>

<a id=03-save-file></a>
## Save Processed File

In [13]:
# Enter code here

<a href=#toc>Back to the top</a>